In [2]:
from kadlu import hycom
import numpy as np
from datetime import datetime

In [3]:
source = hycom()

In [4]:
# parameter selection
start = datetime(2015, 3, 9, 13, 25, 0)
end   = datetime(2015, 3, 9, 19, 0, 0)

# gulf st lawrence
south, west = 44.013, -70.429
north, east = 51.823, -55.977
top, bottom = 0, 0 

In [13]:
source.fetch_salinity(south=south, north=north, west=west, east=east, 
                      start=start, end=end, top=top, bottom=bottom)

downloading 107016 salinity values from hycom...
downloaded in 5.937s. parsed and inserted 46344 rows in 2.182s
60672 null values removed, 0 duplicate rows ignored



In [15]:
source.fetch_temp(south=south, north=north, west=west, east=east, 
                      start=start, end=end, top=top, bottom=bottom)

downloading 107016 water_temp values from hycom...
downloaded in 4.988s. parsed and inserted 46344 rows in 3.611s
60672 null values removed, 0 duplicate rows ignored



In [80]:
from importlib import reload

In [7]:
reload(kadlu.geospatial.data_sources.hycom)

<module 'kadlu.geospatial.data_sources.hycom' from '/home/matt_s/kadlu/kadlu/geospatial/data_sources/hycom.py'>

In [29]:
from kadlu.geospatial.data_sources.fetch_util import database_cfg
from kadlu.geospatial.data_sources.fetch_util import dt_2_epoch

In [7]:
conn, db = database_cfg()

In [111]:
# get temp, salinity values within query boundaries where coordinate
# data exists for both variables
qry = ' AND '.join([
        "SELECT * FROM salinity INNER JOIN water_temp ON salinity.lat = water_temp.lat ",
        "salinity.lon = water_temp.lon ",
        "salinity.time = water_temp.time",
        "salinity.depth = water_temp.depth",
        "salinity.lat >= ?",
        "salinity.lat <= ?",
        "salinity.lon >= ?",
        "salinity.lon <= ?",
        "salinity.time >= ?",
        "salinity.time <= ?",
        "salinity.depth >= ?",
        "salinity.depth <= ? LIMIT 1000000;"])

db.execute(qry, tuple(map(str, [south, north, west, east, dt_2_epoch(start)[0], 
                                dt_2_epoch(end)[0], top, bottom])))
res = np.array(db.fetchall()).T
res

array([['-5521', '-5454', '-5211', ..., '11560', '11631', '11721'],
       ['47.15999984741211', '47.20000076293945', '47.2400016784668',
        ..., '51.720001220703125', '51.7599983215332',
        '51.79999923706055'],
       ['-70.4000244140625', '-70.4000244140625', '-70.4000244140625',
        ..., '-56.0', '-56.0', '-56.0'],
       ...,
       ['133119', '133119', '133119', ..., '133122', '133122', '133122'],
       ['0', '0', '0', ..., '0', '0', '0'],
       ['hycom', 'hycom', 'hycom', ..., 'hycom', 'hycom', 'hycom']],
      dtype='<U32')

In [112]:
salinity = res[0]
temp = res[6]
lat = res[1]
lon = res[2]

In [113]:
len(temp)

30668

In [76]:
from kadlu import chs

In [77]:
bathysource = chs()
bathy = bathysource.load_bathymetry(south=south, north=north, west=west, east=east)

/home/matt_s/kadlu/storage/


IndexError: boolean index did not match indexed array along dimension 1; dimension is 1002 but corresponding boolean dimension is 1001

In [ ]:
bathy = bathysource.load_bathymetry(south=south, north=north, west=west, east=east)

In [71]:
from kadlu.geospatial.ocean import Ocean
from kadlu.sound.sound_speed import SoundSpeed 
from kadlu.geospatial.data_sources.fetch_util import storage_cfg

In [85]:
import kadlu
from kadlu.geospatial.data_sources import chs
reload(kadlu.geospatial.data_sources.chs)

<module 'kadlu.geospatial.data_sources.chs' from '/home/matt_s/kadlu/kadlu/geospatial/data_sources/chs.py'>

In [101]:
bathysource = chs.Chs()
bathy = bathysource.load_bathymetry(south=50, north=53, west=-52, east=-49)

/home/matt_s/kadlu/storage/
Files exist, skipping retrieval...


In [102]:
bathy

(array([], dtype=float64), array([], dtype=float64), array([], dtype=float64))

In [103]:
oceantest = Ocean(salinity=salinity, temp=temp, bathy='CHS')
oceantest.load_bathy(south=south, north=north, west=west, east=east, storage=storage_cfg())

/home/matt_s/kadlu/storage/
boolean index did not match indexed array along dimension 1; dimension is 1002 but corresponding boolean dimension is 1001
boolean index did not match indexed array along dimension 1; dimension is 1002 but corresponding boolean dimension is 1001
boolean index did not match indexed array along dimension 1; dimension is 1002 but corresponding boolean dimension is 1001
boolean index did not match indexed array along dimension 1; dimension is 1002 but corresponding boolean dimension is 1001
boolean index did not match indexed array along dimension 1; dimension is 1002 but corresponding boolean dimension is 1001
boolean index did not match indexed array along dimension 1; dimension is 1002 but corresponding boolean dimension is 1001


boolean index did not match indexed array along dimension 1; dimension is 1002 but corresponding boolean dimension is 1001
boolean index did not match indexed array along dimension 1; dimension is 1002 but corresponding boolean dimension is 1001
boolean index did not match indexed array along dimension 1; dimension is 1002 but corresponding boolean dimension is 1001
boolean index did not match indexed array along dimension 1; dimension is 1002 but corresponding boolean dimension is 1001
boolean index did not match indexed array along dimension 1; dimension is 1002 but corresponding boolean dimension is 1001
boolean index did not match indexed array along dimension 1; dimension is 1002 but corresponding boolean dimension is 1001
boolean index did not match indexed array along dimension 1; dimension is 1002 but corresponding boolean dimension is 1001
boolean index did not match indexed array along dimension 1; dimension is 1002 but corresponding boolean dimension is 1001
boolean index di

ValueError: zero-size array to reduction operation minimum which has no identity

In [68]:
soundtest = SoundSpeed(ocean=oceantest )

AssertionError: Bathymetric data have not been loaded